In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

This is a simple implement of stock trading stragety. We first use Arima model to predict the next days stock close price. Then we make trading decisions based on predicted trend changes.

In [2]:
# Load data
data = pd.read_csv('Tape.csv')

# Using Arima model to predict future stock prices
model = ARIMA(data['Close'], order=(5, 1, 0))
model_fit = model.fit()
forecast = model_fit.forecast(steps=25)  # Forecasting the next 25 days

The trading stragety is:

Buy condition: If the predicted price shows an upward trend in the next few days (for example, a continuous 3-day prediction of price increase), then buy at the beginning of the first day.

Sell condition: If holding the stock, the predicted price shows a downward trend in the next few days (for example, a continuous 3-day prediction of price decrease), then sell at the beginning of the first day.

In [3]:
# Trading strategy
profit = 0.0
is_holding = False
buy_price = 0.0
trend_days = 3  # Trend is determined by the difference of the last 3 days

for i in range(len(forecast) - trend_days):
    trend = forecast[i+1:i+trend_days+1].diff().dropna()
    
    if not is_holding and all(trend > 0):
        # If not holding and forecast is trending up, buy
        is_holding = True
        buy_price = forecast.iloc[i]
    elif is_holding and all(trend < 0):
        # If holding and forecast is trending down, sell
        profit += forecast.iloc[i] - buy_price
        is_holding = False
        buy_price = 0.0  # Reset buy price

In [4]:
# Sell remaining stocks if still holding
if is_holding:
    profit += forecast.iloc[-1] - buy_price


In [5]:
print(f"Estimated Profit: {profit}")

Estimated Profit: 3.8365609204326745
